In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the image size and number of classes
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 4

# Load labeled data
labeled_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
labeled_train_set = labeled_datagen.flow_from_directory(
    '/content/drive/MyDrive/LungXRays-grayscale/train',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical'
)

# Load unlabeled data
unlabeled_datagen = ImageDataGenerator(rescale=1./255)
unlabeled_train_set = unlabeled_datagen.flow_from_directory(
    '/content/drive/MyDrive/LungXRays-grayscale/unlabeled',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode=None,
    shuffle=False
)

# Load and prepare the test dataset
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/LungXRays-grayscale/test',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical'
)

# Create the base VGG16 model
base_model = VGG16(
    input_shape=(*IMAGE_SIZE, 3),
    weights='imagenet',
    include_top=False
)

# Freeze the layers of the pre-trained VGG16 model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the semi-supervised model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)



Found 4649 images belonging to 4 classes.
Found 0 images belonging to 0 classes.
Found 1552 images belonging to 4 classes.
58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Train the model on labeled data
model.fit(
    labeled_train_set,
    validation_data=test_set,
    steps_per_epoch=50,
    epochs=10
)


Epoch 1/10
50/50 [==============================] - 2040s 40s/step - loss: 1.2777 - accuracy: 0.4269 - val_loss: 0.8018 - val_accuracy: 0.8222
Epoch 2/10
50/50 [==============================] - 2016s 40s/step - loss: 0.9015 - accuracy: 0.6338 - val_loss: 0.5289 - val_accuracy: 0.8789
Epoch 3/10
50/50 [==============================] - 1950s 39s/step - loss: 0.7932 - accuracy: 0.6937 - val_loss: 0.4300 - val_accuracy: 0.8898
Epoch 4/10
50/50 [==============================] - 1950s 39s/step - loss: 0.6859 - accuracy: 0.7292 - val_loss: 0.3931 - val_accuracy: 0.8892
Epoch 5/10
50/50 [==============================] - 2006s 40s/step - loss: 0.6304 - accuracy: 0.7625 - val_loss: 0.3366 - val_accuracy: 0.9091
Epoch 6/10
50/50 [==============================] - 1998s 40s/step - loss: 0.5361 - accuracy: 0.8094 - val_loss: 0.3325 - val_accuracy: 0.8905
Epoch 7/10
50/50 [==============================] - 1957s 39s/step - loss: 0.5330 - accuracy: 0.8022 - val_loss: 0.2589 - val_accuracy: 0.9253

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_set)
print("Test Loss:", test_loss*100)
print("Test Accuracy:", test_accuracy*100)

49/49 [==============================] - 946s 19s/step - loss: 0.2072 - accuracy: 0.9343
Test Loss: 20.724283158779144
Test Accuracy: 93.42783689498901


In [ ]:

# Generate pseudo-labels for unlabeled data
unlabeled_predictions = model.predict(unlabeled_train_set)
pseudo_labels = np.argmax(unlabeled_predictions, axis=1)




ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [ ]:

# Add pseudo-labeled samples to the labeled dataset
unlabeled_samples = unlabeled_train_set[0][0]
unlabeled_labels = tf.keras.utils.to_categorical(pseudo_labels, num_classes=NUM_CLASSES)
unlabeled_samples_reshaped = np.reshape(unlabeled_samples, (-1, *IMAGE_SIZE, 3))
combined_images = np.concatenate([labeled_train_set[0][0], unlabeled_samples_reshaped])
combined_labels = np.concatenate([labeled_train_set[0][1], unlabeled_labels])

# Shuffle the combined data
combined_data = list(zip(combined_images, combined_labels))
np.random.shuffle(combined_data)
combined_images, combined_labels = zip(*combined_data)

# Convert to numpy arrays
combined_images = np.array(combined_images)
combined_labels = np.array(combined_labels)



In [ ]:
# Train the model on combined labeled and pseudo-labeled data
model.fit(
    combined_images,
    combined_labels,
    validation_data=test_set,
    epochs=20
)

Epoch 1/20
2/2 [==============================] - 107s 104s/step - loss: 0.2369 - accuracy: 0.9091 - val_loss: 0.3119 - val_accuracy: 0.9196
Epoch 2/20
2/2 [==============================] - 111s 106s/step - loss: 0.1703 - accuracy: 0.8788 - val_loss: 0.3286 - val_accuracy: 0.9144
Epoch 3/20
2/2 [==============================] - 110s 105s/step - loss: 0.2504 - accuracy: 0.9394 - val_loss: 0.3686 - val_accuracy: 0.9066
Epoch 4/20
2/2 [==============================] - 110s 106s/step - loss: 0.1360 - accuracy: 0.9091 - val_loss: 0.4268 - val_accuracy: 0.8859
Epoch 5/20
2/2 [==============================] - 108s 104s/step - loss: 0.1235 - accuracy: 0.9697 - val_loss: 0.4974 - val_accuracy: 0.8534
Epoch 6/20
2/2 [==============================] - 109s 105s/step - loss: 0.1404 - accuracy: 0.9394 - val_loss: 0.5551 - val_accuracy: 0.8418
Epoch 7/20
2/2 [==============================] - 110s 106s/step - loss: 0.0728 - accuracy: 1.0000 - val_loss: 0.6139 - val_accuracy: 0.8119
Epoch 8/20
2/

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_set)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy*100)


25/25 [==============================] - 106s 4s/step - loss: 0.3172 - accuracy: 0.9157
Test Loss: 0.317180871963501
Test Accuracy: 91.56938791275024


In [ ]:
# Example image prediction
img_path = '/Users/praveenmahan/Desktop/sunny/train/pneumonia/person7_bacteria_24.jpeg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = tf.keras.applications.vgg19.preprocess_input(x)

In [ ]:
# Predict the class of the image
class_probs = model.predict(x)

1/1 [==============================] - 0s 138ms/step


In [ ]:
# Define class labels
class_labels = ['Corona Virus Disease', 'Normal', 'Pneumonia', 'Tuberculosis']
predicted_class_index = np.argmax(class_probs)
predicted_class_label = class_labels[predicted_class_index]
print(predicted_class_label)

Pneumonia


In [ ]:
# Example image prediction
img_path = '/Users/praveenmahan/Desktop/untrain/1/COVID19(385).jpg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = tf.keras.applications.vgg19.preprocess_input(x)
# Predict the class of the image
class_probs = model.predict(x)
# Define class labels
class_labels = ['Corona Virus Disease', 'Normal', 'Pneumonia', 'Tuberculosis']
predicted_class_index = np.argmax(class_probs)
predicted_class_label = class_labels[predicted_class_index]
print(predicted_class_label)

1/1 [==============================] - 1s 785ms/step
Corona Virus Disease


In [ ]:
# Example image prediction
img_path = '/Users/praveenmahan/Desktop/tb.png'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = tf.keras.applications.vgg19.preprocess_input(x)
# Predict the class of the image
class_probs = model.predict(x)
# Define class labels
class_labels = ['Corona Virus Disease', 'Normal', 'Pneumonia', 'Tuberculosis']
predicted_class_index = np.argmax(class_probs)
predicted_class_label = class_labels[predicted_class_index]
print(predicted_class_label)

1/1 [==============================] - 1s 517ms/step
Tuberculosis


In [ ]:
# Example image prediction
img_path = '/Users/praveenmahan/Desktop/mp/val/PNEUMONIA/person1947_bacteria_4876.jpeg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = tf.keras.applications.vgg19.preprocess_input(x)
# Predict the class of the image
class_probs = model.predict(x)
# Define class labels
class_labels = ['Corona Virus Disease', 'Normal', 'Pneumonia', 'Tuberculosis']
predicted_class_index = np.argmax(class_probs)
predicted_class_label = class_labels[predicted_class_index]
print(predicted_class_label)

1/1 [==============================] - 1s 755ms/step
Pneumonia
